# Config

In [19]:
from hbconfig import Config
Config("./config.yml")
Config

Read config file name: ./config
{
    "data": {
        "type": "all",
        "base_path": "data/",
        "processed_path": "processed_all_dialogs_data",
        "word_threshold": 6,
        "max_seq_length": 200,
        "sentence_diff": 0.33,
        "testset_size": 50000,
        "PAD_ID": 0,
        "UNK_ID": 1,
        "START_ID": 2,
        "EOS_ID": 3
    },
    "model": {
        "batch_size": 32,
        "num_layers": 4,
        "num_units": 512,
        "embed_dim": 300,
        "embed_share": true,
        "cell_type": "gru",
        "dropout": 0.2,
        "encoder_type": "bi",
        "attention_mechanism": "bahdanau"
    },
    "train": {
        "learning_rate": 0.001,
        "sampling_probability": 0.4,
        "train_steps": 100000,
        "model_dir": "logs/all_dialogs",
        "save_checkpoints_steps": 1000,
        "loss_hook_n_iter": 1000,
        "check_hook_n_iter": 1000,
        "min_eval_frequency": 1000,
        "print_verbose": true,
        "debug": fa

# Encoding

In [3]:
import tensorflow as tf

#__all__ = ["Encoder"]

class Encoder:
    """Encoder class is Mutil-layer Recurrent Neural Networks
    The 'Encoder' usually encode the sequential input vector.
    """
    
    UNI_ENCODER_TYPE = "uni"
    BI_ENCODER_TYPE = "bi"
    
    RNN_GRU_CELL = "gru"
    RNN_LSTM_CELL = "lstm"
    RNN_NAS_CELL = "nas"
    RNN_LAYER_NORM_LSTM_CELL = "layer_norm_lstm"
    
    def __init__(self, 
                 encoder_type="uni",
                 num_layers=4,
                 cell_type="GRU",
                 num_units=512,
                 dropout=0.8,
                 dtype=tf.float32):
        """Contructs an 'Encoder' instance.
        * Args:
            encoder_type: rnn encoder_type (uni, bi)
            num_layers: number of RNN cell composed sequentially of multiple simple cells.
            input_vector: RNN Input vectors.
            sequence_length: batch element's sequence length
            cell_type: RNN cell types (lstm, gru, layer_norm_lstm, nas)
            num_units: the number of units in cell
            dropout: set prob operator adding dropout to inputs of the given cell.
            dtype: the dtype of the input
        * Returns:
            Encoder instance
        """
        self.encoder_type = encoder_type
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.num_units = num_units
        self.dropout = dropout
        self.dtype = dtype
        
    def build(self, input_vector, sequence_length, scope=None):
        if self.encoder_type == self.UNIT_ENCODER_TYPE:
            self.cells = self._create_rnn_clls()
            return self.unidirectional_rnn(input_vector, sequence_length, scope=scope)
        
        elif self.encoder_type == self.BI_ENCODER_TYPE:
            if self.num_layers == 0:
                self.num_layers = 1
            self.cells_fw = self._create_rnn_cells(is_list=True)
            self.cells_bw = self._create_rnn_cells(is_list=True)
            
    def _create_rnn_cells(self, is_list=False):
        """Contructs stacked_rnn with num_layers
        * Args:
            is_list: flags for stack bidirectional. True=stack bidirectional, False=unidirectional
        * Returns:
            stacked_rnn
        """
        stacked_rnn = []
        for _ in range(self.num_layers):
            single_cell = self._rnn_single_cell()
            stacked_rnn.append(single_cell)
            
        if is_list:
            return stacked_rnn
        else:
            return tf.nn.rnn_cell.MultiRNNCell(
                                  cells=stacked_rnn,
                                  state_is_tuple=True)
        
        
    def _rnn_single_cell(self):
        """Contructs rnn single_cell"""

        if self.cell_type == self.RNN_GRU_CELL:
            single_cell = tf.contrib.rnn.GRUCell(
                self.num_units,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_LSTM_CELL:
            single_cell = tf.contrib.rnn.BasicLSTMCell(
                self.num_units,
                forget_bias=1.0,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_LAYER_NORM_LSTM_CELL:
            single_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.num_units,
                forget_bias=1.0,
                layer_norm=True,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_NAS_CELL:
            single_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.num_units)
        else:
            raise ValueError(f"Unknown rnn cell type. {self.cell_type}")

        if self.dropout > 0.0:
            single_cell = tf.contrib.rnn.DropoutWrapper(
                cell=single_cell, input_keep_prob=(1.0 - self.dropout))
        return single_cell

# Attention

In [7]:
# import tensorflow as tf
# __all__ = ["Attention", "Decoder"]


class Attention:
    """Attention class """
    
    BAHDANAU_MECHANISM = "bahdanau"
    NORMED_BAHDANAU_MECHANISM = "normed_bahdanau"

    LUONG_MECHANISM = "luong"
    SCALED_LUONG_MECHANISM = "scaled_luong"
    
    def __init__(self,
                 attention_mechanism="bahdanau",
                 encoder_type="bi",
                 num_units=512,
                 memory=None,
                 memory_sequence_length=None):
        
        assert memory is not None
        assert memory_sequence_length is not None
        
        self.attention_mechanism = attention_mechanism
        self.encoder_type = encoder_type
        self.num_units = num_units
        self.memory = memory
        self.memory_sequence_length = memory_sequence_length
        
    
    def wrap(self, decoder_cell, alignment_history=True):
        with tf.variable_scope("attention") as scope:
            attention_layer_size = self.num_units
            return tf.contrib.seq2seq.AttentionWrapper(
                        decoder_cell,
                        self._create_mechanism(),
                        attention_layer_size=attention_layer_size,
                        alignment_history=alignment_history,
                        name=f"{self.attention_mechanism}-mechanism")
        
        
    
    def _create_mechanism(self):

        if self.attention_mechanism == "bahdanau":
            return tf.contrib.seq2seq.BahdanauAttention(
                    self.num_units,
                    self.memory,
                    memory_sequence_length=self.memory_sequence_length)

        elif self.attention_mechanism == "normed_bahdanau":
            return tf.contrib.seq2seq.BahdanauAttention(
                    self.num_units,
                    self.memory,
                    memory_sequence_length=self.memory_sequence_length,
                    normalize=True)

        elif self.attention_mechanism == "luong":
            return tf.contrib.seq2seq.LuongAttention(
                    self.num_units,
                    self.memory,
                    memory_sequence_length=self.memory_sequence_length)

        elif self.attention_mechanism == "scaled_luong":
            return tf.contrib.seq2seq.LuongAttention(
                    self.num_units,
                    self.memory,
                    memory_sequence_length=self.memory_sequence_length,
                    scale=True)
        else:
            raise ValueError(f"Unknown attention mechanism {self.attention_mechanism}")

# Decoder Model

In [8]:
class Decoder:
    """Decoder class"""

    UNI_ENCODER_TYPE = "uni"
    BI_ENCODER_TYPE = "bi"

    RNN_GRU_CELL = "gru"
    RNN_LSTM_CELL = "lstm"
    RNN_LAYER_NORM_LSTM_CELL = "layer_norm_lstm"
    RNN_NAS_CELL = "nas"

    def __init__(self,
                 cell_type="lstm",
                 dropout=0.8,
                 encoder_type="uni",
                 num_layers=None,
                 num_units=None,
                 sampling_probability=0.4,
                 mode=tf.estimator.ModeKeys.TRAIN,
                 dtype=tf.float32):

        self.cell_type = cell_type
        self.dropout = dropout
        self.encoder_type = encoder_type
        self.num_layers = num_layers
        self.num_units = num_units
        self.sampling_probability = sampling_probability

        if encoder_type == self.BI_ENCODER_TYPE:
            self.num_units *= 2
            self.num_layers = int(self.num_layers / 2)
            if self.num_layers == 0:
                self.num_layers = 1
        self.mode = mode
        self.dtype = dtype

        
    def set_attention_then_project(self,
                                   attention_mechanism="bahdanau",
                                   beam_width=0,
                                   memory=None,
                                   memory_sequence_length=None,
                                   vocab_size=None):

        self.beam_width = beam_width

        cells = self._create_rnn_cells()

        attention = Attention(
                        attention_mechanism=attention_mechanism,
                        encoder_type=self.encoder_type,
                        num_units=self.num_units,
                        memory=memory,
                        memory_sequence_length=memory_sequence_length)
        alignment_history = (self.mode == tf.estimator.ModeKeys.PREDICT
                and self.beam_width == 0)

        attn_cell = attention.wrap(cells, alignment_history=alignment_history)
        self.out_cell = tf.contrib.rnn.OutputProjectionWrapper(
            attn_cell, vocab_size)

        self.maximum_iterations = tf.round(tf.reduce_max(memory_sequence_length) * 2)

    def set_initial_state(self, batch_size, encoder_final_state):
        if self.mode == tf.estimator.ModeKeys.PREDICT and self.beam_width > 0:
            decoder_start_state = tf.contrib.seq2seq.tile_batch(encoder_final_state, self.beam_width)
            self.decoder_initial_state = self.out_cell.zero_state(batch_size * self.beam_width, self.dtype)
            self.decoder_initial_state = self.decoder_initial_state.clone(cell_state=decoder_start_state)
        else:
            self.decoder_initial_state = self.out_cell.zero_state(batch_size, self.dtype)
            self.decoder_initial_state = self.decoder_initial_state.clone(cell_state=encoder_final_state)

            
            
    def build(self,
            inputs=None,
            sequence_length=None,
            embedding=None,
            start_tokens=None,
            end_token=None,
            length_penalty_weight=1.0):

        if self.mode == tf.estimator.ModeKeys.TRAIN:
            assert inputs is not None
            assert sequence_length is not None

            helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
                    inputs=inputs,
                    sequence_length=sequence_length,
                    embedding=embedding,
                    sampling_probability=self.sampling_probability)

            return self._basic_decoder(helper)

        else:
            assert embedding is not None
            assert start_tokens is not None
            assert end_token is not None

            if self.mode == tf.estimator.ModeKeys.PREDICT and self.beam_width > 0:
                return self._beam_search_decoder(
                        embedding, start_tokens, end_token, length_penalty_weight)
            else:
                helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                        embedding=embedding,
                        start_tokens=start_tokens,
                        end_token=end_token)
                return self._basic_decoder(helper)
            

    def _basic_decoder(self, helper):
        decoder = tf.contrib.seq2seq.BasicDecoder(
            cell=self.out_cell,
            helper=helper,
            initial_state=self.decoder_initial_state)

        if self.mode == tf.estimator.ModeKeys.TRAIN:
            outputs, final_context_state, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder,
                output_time_major=False,
                impute_finished=True,
                swap_memory=True)
        else:
            outputs, final_context_state, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder,
                output_time_major=False,
                impute_finished=True,
                maximum_iterations=self.maximum_iterations)

        return outputs

    
    
    def _beam_search_decoder(self, embedding, start_tokens, end_token, length_penalty_weight):
        decoder = tf.contrib.seq2seq.BeamSearchDecoder(
            cell=self.out_cell,
            embedding=embedding,
            start_tokens=start_tokens,
            end_token=end_token,
            initial_state=self.decoder_initial_state,
            beam_width=self.beam_width,
            length_penalty_weight=length_penalty_weight)

        outputs, final_context_state, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder=decoder,
            output_time_major=False,
            impute_finished=False,
            maximum_iterations=self.maximum_iterations)
        return outputs
    
    

    def _create_rnn_cells(self):
        """Contructs stacked_rnn with num_layers
        * Args:
            is_list: flags for stack bidirectional. True=stack bidirectional, False=unidirectional
        * Returns:
            stacked_rnn
        """

        stacked_rnn = []
        for _ in range(self.num_layers):
            single_cell = self._rnn_single_cell()
            stacked_rnn.append(single_cell)

        if self.num_layers == 1:
            return stacked_rnn[0]
        else:
            return tf.nn.rnn_cell.MultiRNNCell(
                    cells=stacked_rnn,
                    state_is_tuple=True)
        
        

    def _rnn_single_cell(self):
        """Contructs rnn single_cell"""

        if self.cell_type == self.RNN_GRU_CELL:
            single_cell = tf.contrib.rnn.GRUCell(
                self.num_units,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_LSTM_CELL:
            single_cell = tf.contrib.rnn.BasicLSTMCell(
                self.num_units,
                forget_bias=1.0,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_LAYER_NORM_LSTM_CELL:
            single_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.num_units,
                forget_bias=1.0,
                layer_norm=True,
                reuse=tf.get_variable_scope().reuse)
        elif self.cell_type == self.RNN_NAS_CELL:
            single_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.num_units)
        else:
            raise ValueError(f"Unknown rnn cell type. {self.cell_type}")

        if self.dropout > 0.0:
            single_cell = tf.contrib.rnn.DropoutWrapper(
                cell=single_cell, input_keep_prob=(1.0 - self.dropout))

        return single_cell

# Graph : combine Encoder & Decoder Model

In [ ]:
class Graph:

    def __init__(self, mode=None, dtype=tf.float32):
        self.mode = mode
        self.beam_width = Config.predict.get('beam_width', 0)
        self.dtype = dtype

    def build(self,
              encoder_inputs=None,
              decoder_inputs=None):

        # set inputs variable
        self.encoder_inputs = encoder_inputs
        self.encoder_input_lengths = tf.reduce_sum(
                     tf.to_int32(tf.not_equal(self.encoder_inputs, Config.data.PAD_ID)), 1,
                     name="encoder_input_lengths")

        if self.mode == tf.estimator.ModeKeys.TRAIN or self.mode == tf.estimator.ModeKeys.EVAL:
            self.decoder_inputs = decoder_inputs
            self.decoder_input_lengths = tf.reduce_sum(
                tf.to_int32(tf.not_equal(self.decoder_inputs, Config.data.PAD_ID)), 1,
                name="decoder_input_lengths")
        else:
            self.decoder_inputs = None
            self.decoder_input_lengths = None

        # self._build_embed()  # 不需要 Embed
        self._build_encoder()
        self._build_decoder()


    def _build_embed(self):
        with tf.variable_scope ("embeddings", dtype=self.dtype) as scope:
            self.embedding_decoder = tf.get_variable(
                       "embedding_decoder", [Config.data.vocab_size, Config.model.embed_dim], self.dtype)
            if self.mode == tf.estimator.ModeKeys.TRAIN:
                self.decoder_emb_inp = tf.nn.embedding_lookup(
                    self.embedding_decoder, self.decoder_inputs)
            else:
                self.decoder_emb_inp=None
            
        
'''
        with tf.variable_scope ("embeddings", dtype=self.dtype) as scope:
            if Config.model.embed_share:
                embedding = tf.get_variable(
                    "embedding_share", [Config.data.vocab_size, Config.model.embed_dim], self.dtype)

                self.embedding_encoder = embedding
                self.embedding_decoder = embedding
            else:
                self.embedding_encoder = tf.get_variable(
                    "embedding_encoder", [Config.data.vocab_size, Config.model.embed_dim], self.dtype)
            self.embedding_decoder = tf.get_variable(
                    "embedding_decoder", [Config.data.vocab_size, Config.model.embed_dim], self.dtype)

            self.encoder_emb_inp = tf.nn.embedding_lookup(
                self.embedding_encoder, self.encoder_inputs)

            if self.mode == tf.estimator.ModeKeys.TRAIN:
                self.decoder_emb_inp = tf.nn.embedding_lookup(
                    self.embedding_decoder, self.decoder_inputs)
            else:
                self.decoder_emb_inp=None
'''



    def _build_encoder(self):
        with tf.variable_scope('encoder'):
            encoder = Encoder(
                        encoder_type = Config.model.encoder_type,
                        num_layers = Config.model.num_layers,
                        cell_type = Config.model.cell_type,
                        num_units = Config.model.num_units,
                        dropout = Config.model.dropout)

            self.encoder_outputs, self.encoder_final_state = encoder.build(
                                  input_vector = self.encoder_inputs #self.encoder_emb_inp,
                                  sequence_length = self.encoder_input_lengths)

            if self.mode == tf.estimator.ModeKeys.PREDICT and self.beam_width > 0:
                self.encoder_outputs = tf.contrib.seq2seq.tile_batch(
                                          self.encoder_outputs, self.beam_width )
                self.encoder_input_lengths = tf.contrib.seq2seq.tile_batch(
                                          self.encoder_input_lengths, self.beam_width )

    def _build_decoder(self):

        batch_size = tf.shape(self.encoder_inputs)[0]

        with tf.variable_scope('decoder'):

            decoder = Decoder(
                        cell_type = Config.model.cell_type,
                        dropout = Config.model.dropout,
                        encoder_type = Config.model.encoder_type,
                        num_layers = Config.model.num_layers,
                        num_units = Config.model.num_units,
                        sampling_probability = Config.train.sampling_probability,
                        mode = self.mode,
                        dtype = self.dtype)

            decoder.set_attention_then_project(
                        attention_mechanism=Config.model.attention_mechanism,
                        beam_width=self.beam_width,
                        memory=self.encoder_outputs,
                        memory_sequence_length=self.encoder_input_lengths,
                        vocab_size=Config.data.vocab_size)
            decoder.set_initial_state(batch_size, self.encoder_final_state)

            decoder_outputs = decoder.build(
                                inputs=self.decoder_inputs #self.decoder_emb_inp,
                                sequence_length=self.decoder_input_lengths,
                                embedding=self.embedding_decoder,
                                start_tokens=tf.fill([batch_size], Config.data.START_ID),
                                end_token=Config.data.EOS_ID,
                                length_penalty_weight=Config.predict.length_penalty_weight)

            if self.mode == tf.estimator.ModeKeys.TRAIN:
                self.decoder_logits = decoder_outputs.rnn_output
            else:
                if self.mode == tf.estimator.ModeKeys.PREDICT and self.beam_width > 0:
                    self.decoder_logits = tf.no_op()
                    self.predictions = decoder_outputs.predicted_ids
                else:
                    self.decoder_logits = decoder_outputs.rnn_output
                    self.predictions = decoder_outputs.sample_id

            if self.mode == tf.estimator.ModeKeys.PREDICT:
                # PREDICT mode do not need loss
                return

            decoder_output_length = tf.shape(self.decoder_logits)[1]

            def concat_zero_padding():
                pad_num = Config.data.max_seq_length - decoder_output_length
                zero_padding = tf.zeros(
                        [batch_size, pad_num, Config.data.vocab_size],
                        dtype=self.dtype)

                return tf.concat([self.decoder_logits, zero_padding], axis=1)

            def slice_to_max_len():
                return tf.slice(self.decoder_logits,
                                [0, 0, 0],
                                [batch_size, Config.data.max_seq_length, Config.data.vocab_size])

            # decoder output sometimes exceed max_seq_length
            self.logits = tf.cond(decoder_output_length < Config.data.max_seq_length,
                                  concat_zero_padding,
                                  slice_to_max_len)
            self.predictions = tf.argmax(self.logits, axis=2)

            self.weight_masks = tf.sequence_mask(
                lengths=self.decoder_input_lengths,
                maxlen=Config.data.max_seq_length,
                dtype=self.dtype, name='masks')

        if self.mode == tf.estimator.ModeKeys.TRAIN:
            self.train_predictions = tf.argmax(self.logits, axis=2)
            # for print trainig data
            tf.identity(tf.argmax(self.decoder_logits[0], axis=1), name='train/pred_0')

# seq2seq Model

In [ ]:
# import tensorflow as tf

class seq2seq:
    def __init__():
        pass
    
    def model_fn(self, mode, features, labels, params):
        self.dtype = tf.float32
        self.mode = mode
        
        self.loss, self.train_op, self.metrics, self.predictions = None, None, None, None
        # initializing train data
        self._init_placeholder(features, labels)
        #
        self.build_graph()

        # train mode: required loss and train_op
        # eval mode: required loss
        # predict mode: required predictions

        return tf.estimator.EstimatorSpec(
                                    mode = mode,
                                    loss = self.loss,
                                    train_op = self.train_op,
                                    eval_metric_ops = self.metrics,
                                    predictions = {"prediction": self.predictions})
  


    def _init_placeholder(self, features, labels):
        
        ''' Declare Encoder input data'''
        self.encoder_inputs = features
        if type(features) == dict:
            self.encoder_inputs = features["input_data"]
  
        ''' '''
        batch_size = tf.shape(self.encoder_inputs)[0]
        
        
        if self.mode == tf.estimator.ModeKeys.TRAIN or self.mode == tf.estimator.ModeKeys.EVAL:
            ''' Declare Decoder label data'''
            self.decoder_inputs = labels
            decoder_input_shift_1 = tf.slice(self.decoder_inputs, 
                                             [0, 1], #begin
                                             [batch_size, Config.data.max_seq_length-1]) # size
            pad_tokens = tf.zeros([batch_size, 1], dtype=tf.int32)

            # make target (right shift 1 from decoder_inputs)
            self.targets = tf.concat([decoder_input_shift_1, pad_tokens], axis=1)
        else:
            self.decoder_inputs = None
            

            

    def build_graph(self):
        graph = seq2seq_attention.Graph(mode=self.mode) # call encoder & decoder model
        graph.build(encoder_inputs=self.encoder_inputs,
                    decoder_inputs=self.decoder_inputs)

        self.predictions = graph.predictions
        if self.mode != tf.estimator.ModeKeys.PREDICT:
            self._build_loss(graph.logits, graph.weight_masks)
            self._build_optimizer()
            self._build_metric()
            
            

    def _build_loss(self, logits, weight_masks):
        self.loss = tf.contrib.seq2seq.sequence_loss(
                logits=logits,
                targets=self.targets,
                weights=weight_masks,
                name="loss")
        
        

    def _build_optimizer(self):
        self.train_op = tf.contrib.layers.optimize_loss(
            self.loss, tf.train.get_global_step(),
            optimizer='Adam',
            learning_rate=Config.train.learning_rate,
            summaries=['loss', 'learning_rate'],
            name="train_op")
        
        

    def _build_metric(self):

        def blue_score(labels, predictions,
                       weights=None, metrics_collections=None,
                       updates_collections=None, name=None):

            def _nltk_blue_score(labels, predictions):

                # slice after <eos>
                predictions = predictions.tolist()
                for i in range(len(predictions)):
                    prediction = predictions[i]
                    if Config.data.EOS_ID in prediction:
                        predictions[i] = prediction[:prediction.index(Config.data.EOS_ID)+1]

                labels = [
                    [[w_id for w_id in label if w_id != Config.data.PAD_ID]]
                    for label in labels.tolist()]
                predictions = [
                    [w_id for w_id in prediction]
                    for prediction in predictions]

                return float(nltk.translate.bleu_score.corpus_bleu(labels, predictions))

            score = tf.py_func(_nltk_blue_score, (labels, predictions), tf.float64)
            return tf.metrics.mean(score * 100)

        self.metrics = {
            "accuracy": tf.metrics.accuracy(self.targets, self.predictions),
            "bleu": blue_score(self.targets, self.predictions)
        }

# Main function

In [22]:
import argparse
import logging

from hbconfig import Config
import tensorflow as tf

In [25]:
def experiment_fn(run_config, params):

    conversation = seq2seq() # Declare S2S model
    
    
    estimator = tf.estimator.Estimator(
                   model_fn=seq2seq.model_fn,
                   model_dir=Config.train.model_dir, # 
                   params=params,
                   config=run_config)
    
    
    #Loading data
    vocab = data_loader.load_data('data.csv')
    
    
    Config.data.vocab_size = len(vocab)
    
    # spliting train & test data
    train_X, test_X, train_y, test_y = data_loader.make_train_and_test_set()

    # batch_size
    train_input_fn, train_input_hook = data_loader.make_batch((train_X, train_y), batch_size=Config.model.batch_size)
    test_input_fn, test_input_hook = data_loader.make_batch((test_X, test_y), batch_size=Config.model.batch_size, scope="test")

    
    train_hooks = [train_input_hook]
    if Config.train.print_verbose:
        train_hooks.append(hook.print_variables(
            variables=['train/enc_0', 'train/dec_0', 'train/pred_0'],
            rev_vocab=utils.get_rev_vocab(vocab),
            every_n_iter=Config.train.check_hook_n_iter))
    if Config.train.debug:
        train_hooks.append(tf_debug.LocalCLIDebugHook())

    eval_hooks = [test_input_hook]
    if Config.train.debug:
        eval_hooks.append(tf_debug.LocalCLIDebugHook())

    experiment = tf.contrib.learn.Experiment(
        estimator=estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=test_input_fn,
        train_steps=Config.train.train_steps,
        min_eval_frequency=Config.train.min_eval_frequency,
        train_monitors=train_hooks,
        eval_hooks=eval_hooks,
        eval_delay_secs=0
    )
    return experiment

In [29]:
def main(mode):
    params = tf.contrib.training.HParams(**Config.model.to_dict())

    run_config = tf.contrib.learn.RunConfig(
                      model_dir=Config.train.model_dir,
                      save_checkpoints_steps=Config.train.save_checkpoints_steps)

    tf.contrib.learn.learn_runner.run(
                       experiment_fn=experiment_fn,
                       run_config=run_config,
                       schedule=mode,
                       hparams=params
                                     )


'''
if __name__ == '__main__':
    main('train')
'''

"\nif __name__ == '__main__':\n    main('train')\n"

# Data Preprocessing

In [1]:
from __future__ import print_function

import os
import re
import random
import numpy as np
import pandas as pd

"""
from hbconfig import Config
import numpy as np
import tensorflow as tf
from tqdm import tqdm
""" 

class data_loader:

    def make_dir(self, path):
        """ Create a directory if there isn't one."""
        try:
            os.mkdir(path)
        except:
            pass



    def load_data(self, file_name):
        print("load data ...")
        data = pd.read_csv(file_name) # ｘ,ｙ　
        

        
        
    def make_batch(data, buffer_size=10000, batch_size=64, scope="train"):
        
        class IteratorInitializerHook(tf.train.SessionRunHook):
            """Hook to initialise data iterator after Session is created."""
            def __init__(self):
                super(IteratorInitializerHook, self).__init__()
                self.iterator_initializer_func = None
            def after_create_session(self, session, coord):
                """Initialise the iterator after the session has been created."""
                self.iterator_initializer_func(session)
        
        def get_inputs():
            iterator_initializer_hook = IteratorInitializerHook()

            def train_inputs():
                with tf.name_scope(scope): # scope = "train"
                    X, y = data # <<<<====

                    # Define placeholders
                    input_placeholder = tf.placeholder(
                                             tf.int32, [None, Config.data.max_seq_length])
                    output_placeholder = tf.placeholder(
                                             tf.int32, [None, Config.data.max_seq_length])

                    # Build dataset iterator
                    dataset = tf.data.Dataset.from_tensor_slices(
                                              (input_placeholder, output_placeholder)
                                                                )

                    if scope == "train":
                        dataset = dataset.repeat(None)  # Infinite iterations
                    else:
                        dataset = dataset.repeat(1)  # 1 Epoch
                        
                    # dataset = dataset.shuffle(buffer_size=buffer_size)
                    dataset = dataset.batch(batch_size)

                    iterator = dataset.make_initializable_iterator()
                    next_X, next_y = iterator.get_next()

                    tf.identity(next_X[0], 'enc_0')
                    tf.identity(next_y[0], 'dec_0')

                    # Set runhook to initialize iterator
                    iterator_initializer_hook.iterator_initializer_func = \
                        lambda sess: sess.run(
                            iterator.initializer,
                            feed_dict={input_placeholder: X,
                                       output_placeholder: y})

                    # Return batched (features, labels)
                    return next_X, next_y

            # Return function and hook
            return train_inputs, iterator_initializer_hook

        return get_inputs()



